In [ ]:
!pip install faiss-gpu

In [1]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [2]:
llm = Ollama(model="llama3")
output = StrOutputParser()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a skilled technical writer.",
        ),
        (
            "human", "{user_input}"
        )
    ]
)
chain = prompt | llm | output
response = chain.invoke({"user_input": "how can langsmith help with testing?"})
print(response)

As a technical writer, I've had the pleasure of collaborating with linguists (or "langsmiths" as you call them) on various projects. When it comes to testing, linguists can bring a unique set of skills to the table.

Here are some ways langsmiths can help with testing:

1. **Native speaker feedback**: Langsmiths who are native speakers of the target language can provide invaluable feedback on the accuracy and naturalness of your test scenarios, test cases, or user interface text.
2. **Cultural insight**: A langsmith's cultural background can inform the development of test scenarios that account for regional differences in usage, idioms, or conventions.
3. **Language-specific testing**: Langsmiths are well-versed in the nuances of their native language(s). They can help create test cases that target specific linguistic features, such as grammar, syntax, or formatting.
4. **Error detection and reporting**: When a langsmith is familiar with both the source text and the target text, they c

In [5]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Invoke chain with RAG context
llm = Ollama(model="llama3")
## Load page content
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

## Vector store things
embeddings = OllamaEmbeddings(model="nomic-embed-text")
text_splitter = RecursiveCharacterTextSplitter()
split_documents = text_splitter.split_documents(docs)
vector_store = FAISS.from_documents(split_documents, embeddings)

## Prompt construction
prompt = ChatPromptTemplate.from_template(
    """
            Answer the following question only based on the given context
                                                    
            <context>
            {context}
            </context>
                                                    
            Question: {input}
"""
)

## Retrieve context from vector store
docs_chain = create_stuff_documents_chain(llm, prompt)
retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, docs_chain)

## Winner winner chicken dinner
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(":::ROUND 2:::")
print(response["answer"])

:::ROUND 2:::
Based on the given context, Langsmith can help with testing in several ways:

* Every playground run is logged in the system and can be used to create test cases or compare with other runs.
* Langsmith allows developers to collect more data on how their LLM applications are performing in real-world scenarios through beta testing.
* It supports sending runs to annotation queues, which allow annotators to closely inspect interesting traces and annotate them with respect to different criteria.
* Langsmith enables you to add runs as examples to datasets (from both the project page and within an annotation queue), expanding your test coverage on real-world scenarios.
* It provides a playground environment for rapid iteration and experimentation, allowing you to quickly test out different prompts and models.

These features enable developers to create test cases, compare different versions of their applications, and track regressions/improvements, making it easier to refine and

In [6]:
vector_store.save_local("faiss_index") # Replace faiss_index with name of your choice